###### 0. System check

In [1]:
!nvidia-smi

Wed May 31 08:16:08 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    30W /  70W |      0MiB / 15360MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers datasets evaluate

!pip install ipywidgets
!pip install sklearn

!pip install --upgrade huggingface_hub

!pip install scikit-learn
!pip install --upgrade accelerate

  Using cached sklearn-0.0.post5.tar.gz (3.7 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=9b5fc3ccf2424be7e1d71d1098acf21e5b21c6f5e1365c51dbb1a33a7bff6333
  Stored in directory: /home/lx/.cache/pip/wheels/d7/16/66/2658c4919dbc6cdf8b4642da29d3bbf83de902fc01e2c9a999
Successfully built sklearn


In [18]:
#!apt install git-lfs

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [3]:
!huggingface-cli login --token ${YOUR_HF_TOKEN} 

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /home/lx/.cache/huggingface/token
Login successful


###### 1. Import libraries 

In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np

###### 2. Load dataset and define helper functions

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [6]:
dataset = load_dataset("PolyAI/banking77")

Found cached dataset banking77 (/home/lx/.cache/huggingface/datasets/PolyAI___banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3080
    })
})

In [8]:
dataset["train"][0]

{'text': 'I am still waiting on my card?', 'label': 11}

In [9]:
labels = dataset["train"].features["label"].names
labels[:5]

['activate_my_card',
 'age_limit',
 'apple_pay_or_google_pay',
 'atm_support',
 'automatic_top_up']

In [10]:
label2id = {label: idx for idx, label in enumerate(labels)}
id2label = {idx: label for idx, label in enumerate(labels)}
num_labels = len(labels)

###### 3. Load model and tokenizer and collator

In [11]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

In [12]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /home/lx/.cache/huggingface/datasets/PolyAI___banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b/cache-3eb98eaad63f0c9c.arrow
Loading cached processed dataset at /home/lx/.cache/huggingface/datasets/PolyAI___banking77/default/1.1.0/aec0289529599d4572d76ab00c8944cb84f88410ad0c9e7da26189d31f62a55b/cache-2d07f228ebc49458.arrow


In [13]:
tokenized_dataset["train"][5]

{'text': 'When did you send me my new card?',
 'label': 11,
 'input_ids': [101, 2043, 2106, 2017, 4604, 2033, 2026, 2047, 4003, 1029, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
id2label[11]

'card_arrival'

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

###### 4. Defined  

In [19]:
training_args = TrainingArguments(
    output_dir="banking-intent-distilbert-classifier",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_strategy = "epoch",
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    num_train_epochs=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/lxyuan/banking-intent-distilbert-classifier into local empty directory.


In [ ]:
trainer.train()

In [21]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/256M [00:00<?, ?B/s]

To https://huggingface.co/lxyuan/banking-intent-distilbert-classifier
   8268a4a..8bb4fdd  main -> main

To https://huggingface.co/lxyuan/banking-intent-distilbert-classifier
   8bb4fdd..97fdc1f  main -> main



'https://huggingface.co/lxyuan/banking-intent-distilbert-classifier/commit/8bb4fddcf9e8f6f8696696410043423a1c66ced7'

In [22]:
tokenizer.push_to_hub("banking-intent-distilbert-classifier")

CommitInfo(commit_url='https://huggingface.co/lxyuan/banking-intent-distilbert-classifier/commit/1cf4fbd8341c4087b4acd55874e3e4494f059737', commit_message='Upload tokenizer', commit_description='', oid='1cf4fbd8341c4087b4acd55874e3e4494f059737', pr_url=None, pr_revision=None, pr_num=None)

In [23]:
trainer.evaluate()
# {'eval_loss': 0.2885272204875946, 'eval_accuracy': 0.9243506493506494}

{'eval_loss': 0.2885272204875946, 'eval_accuracy': 0.9243506493506494}

In [24]:
trainer.evaluate(tokenized_dataset["train"])
# {'eval_loss': 0.05775526165962219, 'eval_accuracy': 0.9925022493252025}

{'eval_loss': 0.05775526165962219, 'eval_accuracy': 0.9925022493252025}

In [29]:
from transformers import TextClassificationPipeline

banking_intend_classifier = TextClassificationPipeline(model=trainer.model, tokenizer=tokenizer, device=0)

In [31]:
banking_intend_classifier("How to report lost card?")

[{'label': 'lost_or_stolen_card', 'score': 0.9518502950668335}]